## Plot the pangenome results
- plot the taxonomic boxplots for pigs vs human
- accumulation curve for pig v human

Should be able to make taxonomy boxplots for each taxonomic rank

In [ ]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# ran the pig fastgather --> tax on all 338 SRAs. Subset to get 100 v 100
# pig df, was done on all 399 datasets
# Subset to 100 and save as file for input to compare to human
dfp = pd.read_csv('../results/sourmash//fastgather/tax/pig_339.tax.csv')
# fg does something weird with scaled, when sigs are not same scale as --scaled parameter
dfp['f_weighted_at_rank']= dfp['f_weighted_at_rank']*10
query_100 = dfp['query_name'].unique()
# subset 100 random SRAs and to df. Use randomized df
query_100_select = pd.Series(query_100).sample(100, random_state=42).tolist()
dfp = dfp[dfp['query_name'].isin(query_100_select)]
dfp.to_csv('./pig_100.tax.csv', index=False)

In [ ]:
# Open the dataframes, and clean them 
dfh = pd.read_csv('./human_100.tax.csv')
dfp = pd.read_csv('./pig_100.tax.csv')

# define the taxonomic rank for all functions later
taxrank = 'genus'

# remove unclassified, keep only columns of interest, only keep certain taxonomic rank
dfh = remove_cols(dfh, taxrank)
dfp = remove_cols(dfp, taxrank)

In [ ]:
# ACCUMULATION CURVE
#plot accumulationcurve for taxonomic rank defined above
plot_accum2(dfp, dfh, 'accum.png', 'pig', 'human')

In [ ]:
## Plot abundance weighted fractions for taxonomic rank in human vs pig (in boxplots)
# col for source, so we can plot later
dfp['source'] = 'pig'
dfh['source'] = 'human'

# what genera are most abundant for either 
dfp = split_lineage(dfp, taxrank)
dfh = split_lineage(dfh, taxrank)

# concat df, so have tax for both pigs and humans, with columns that describes what metaG type the species was found in
df = pd.concat([dfh, dfp], ignore_index=True)


In [ ]:
# keep only tax lineages that are in > 95 of the metaGs
pig_ab = calc_num_metag(dfp, taxrank, 95)
human_ab = calc_num_metag(dfh, taxrank, 95)
# combine the top for human and pig
top_genera_combined = set(pig_ab) | set(human_ab)

# Only keep the tax lineages that are in this top
dfp = dfp[dfp[taxrank].isin(top_genera_combined)]
dfh = dfh[dfh[taxrank].isin(top_genera_combined)]
df = pd.concat([dfh, dfp], ignore_index=True)
df = df[['query_name', taxrank, 'source', 'f_weighted_at_rank']]

In [ ]:
## BOXPLOT
# plot the f_weigthed for pigs v humans in a boxplot
compare_boxplot(df, taxrank, 'genus_pig.v.human.png')

# or boxplot just pigs or just human
boxplotmetagperc_1df(dfh, taxrank, 'genus_human.png')

In [ ]:
def remove_cols(df, rank):
    """ Creates dataframe with only taxonomic info of interest. Only keeps gather columns of interest """
    df = df[df['lineage'] != 'unclassified']
    df = df[['query_name', 'rank', 'lineage', 'f_weighted_at_rank']]
    df['lineage'] = df['lineage'].str.strip()
    # for now keep speces level matches
    df = df[df['rank'] == rank]
    return df

# plot accumulation curve for all species recovered in all metaGs
def plot_accum(df, outplot):
    """ Plot one accumulation curve, at specified taxonomic rank. """
    df = df.lineage.value_counts().to_frame().reset_index()
    df.rename(columns={'count':'num_metaG'}, inplace=True)
    df = df[df['num_metaG'] > 0]
    sns.ecdfplot(data=df, x="num_metaG", stat='percent').set(xlabel = "num_metaGs", ylabel= "% species recovered")
    plt.tight_layout()
    plt.savefig(outplot)

# plot both curves, add subject names (human/chicken/pig/cow whatever)
def plot_accum2(df1, df2, outplot, subject1, subject2):
    """ Plot two accumulation curves, for metagenomes from differents species """
    df1 = df1.lineage.value_counts().to_frame().reset_index()
    df2 = df2.lineage.value_counts().to_frame().reset_index()
    df1.rename(columns={'count':'num_metaG'}, inplace=True)
    df2.rename(columns={'count':'num_metaG'}, inplace=True)
    df1 = df1[df1['num_metaG'] > 0]
    df2 = df2[df2['num_metaG'] > 0]
    plt.figure(figsize=(8, 6))
    sns.ecdfplot(data=df1, x="num_metaG", stat='percent', label=subject1)
    sns.ecdfplot(data=df2, x="num_metaG", stat='percent', label=subject2, color='orange')  
    plt.xlabel("num_metaGs")
    plt.ylabel("% species recovered")
    plt.legend()
    plt.tight_layout()
    plt.savefig(outplot)
    plt.show()

def split_lineage(df, rank):
    """ Creates df with for each taxonomic rank the specific columns """
    split_df = df['lineage'].str.split(';', expand=True)
    # Create the appropriate columns based on the rank
    if rank == 'species':
        df[['domain', 'phylum', 'class', 'order', 'family', 'genus', 'species']] = split_df
    elif rank == 'genus':
        df[['domain', 'phylum', 'class', 'order', 'family', 'genus']] = split_df
    elif rank == 'family':
        df[['domain', 'phylum', 'class', 'order', 'family']] = split_df
    else:
        raise ValueError("Rank must be 'genus' or 'family'.")
    
    return df


def calc_num_metag(df, rank, n):
""" Only keeps taxonomic lineages that were found in more than n metagenomes"""
    counts = df[rank].value_counts()
    valid_strings = counts[counts > n].index
    return valid_strings

# boxplot based off highest abundance
def compare_boxplot(df, rank, outplot):
    means = df.groupby(rank)['f_weighted_at_rank'].mean().sort_values()
    ordered_rank = means.index
    counts = df.groupby([rank, 'source']).size().reset_index(name='count')
    plt.figure(figsize=(12, 8))
    sns.set_style("whitegrid")
    ax = sns.boxplot(x=rank, y='f_weighted_at_rank', data=df, order=ordered_rank, hue='source', palette='Set1', showfliers=False)

    # Add text annotations for counts
    for i, phylo_rank in enumerate(ordered_rank):
        for j, source in enumerate(df['source'].unique()):
            count = counts[(counts[rank] == phylo_rank) & (counts['source'] == source)]['count'].values
            if len(count) > 0:
                count = count[0]
                y_pos = df[(df[rank] == phylo_rank) & (df['source'] == source)]['f_weighted_at_rank'].quantile(0.75)
                # Calculate the x position based on the hue
                x_pos = i + (j - 0.5) * 0.2  # Adjust the 0.2 value if needed for better spacing
                ax.text(x_pos, y_pos + 0.05, f'n: {count}', horizontalalignment='center', size='medium', color='black', rotation=90)

    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    plt.tight_layout()
    plt.savefig(outplot)
    plt.show()

# boxplots per rank, how much of the metaG does rank make up?
def boxplotmetagperc_1df(df, rank, outplot):
    counts = df[rank].value_counts()
    valid_strings = counts[counts > 90].index
    filtered_df = df[df[rank].isin(valid_strings)]
    medians = filtered_df.groupby(rank)['f_weighted_at_rank'].mean().sort_values()
    ordered_categories = medians.index
    counts = filtered_df[rank].value_counts().sort_index()

    means = filtered_df.groupby(rank)['f_weighted_at_rank'].mean().sort_values()
    nobs = filtered_df[rank].value_counts().reindex(means.index).values
    nobs = ["n: " + str(x) for x in nobs]
    plt.figure(figsize=(8, 8))  # Adjust size if necessary
    sns.set_style("whitegrid") 
    ax = sns.boxplot(x=rank, y='f_weighted_at_rank', data=filtered_df, showfliers=False,order=ordered_categories)
    ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
    # Add it to the plot
    pos = range(len(nobs))
    for tick,label in zip(pos,ax.get_xticklabels()):
        ax.text(pos[tick],
                means[tick] + 0.1,
                nobs[tick],
                horizontalalignment='center',
                rotation=45,
                size='small',
                color='black',
                weight='semibold')
    plt.tight_layout()
    plt.savefig(outplot)

